#### Preparation and Data Loading

In [1]:
# add root folder of the project to path
import sys
sys.path.insert(0, '../')

In [2]:
# load markers
import numpy as np
points_3dmd = np.load('../data/calibrate/points_3dmd.npy')

import obj3d
pcd_3dmd = obj3d.np2pcd(points_3dmd)

In [3]:
# load kps
import kps
vicon = kps.MarkerSet()
vicon.load_from_vicon('../dataset/6kmh_softbra_8markers_1.csv')
vicon.interp_field()

points_vicon = vicon.get_frame_coord(0)
pcd_vicon = obj3d.np2pcd(points_vicon)

loaded 1 vicon file: ../dataset/6kmh_softbra_8markers_1.csv


In [4]:
# plot point clouds before calibration
import pyvista as pv
pv.set_jupyter_backend('pythreejs')

pvpcd_3dmd = obj3d.np2pvpcd(points_3dmd, radius=10, phi_resolution=10, theta_resolution=10)

pvpcd_vicon = obj3d.np2pvpcd(points_vicon, radius=10, phi_resolution=10, theta_resolution=10)

scene = pv.Plotter()
scene.add_mesh(pvpcd_3dmd, color='Gold')
scene.add_mesh(pvpcd_vicon, color='Blue')
scene.add_axes(box=True)
scene.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

#### Pre-alignment and Calibration

In [5]:
# pre-alignment
import copy
pcd_3dmd_pre = copy.deepcopy(pcd_3dmd)
r_pre = pcd_3dmd_pre.get_rotation_matrix_from_xyz((np.pi/2, 0, 0))
pcd_3dmd_pre.rotate(r_pre, center=(0, 0, 0))
points_3dmd_pre = obj3d.pcd2np(pcd_3dmd_pre)

In [6]:
# plot point clouds after pre-alignment
pvpcd_3dmd_pre = obj3d.np2pvpcd(points_3dmd_pre, radius=10, phi_resolution=10, theta_resolution=10)

scene = pv.Plotter()
scene.add_mesh(pvpcd_3dmd_pre, color='Green')
scene.add_mesh(pvpcd_3dmd, color='Gold')
scene.add_mesh(pvpcd_vicon, color='Blue')
scene.add_axes(box=True)
scene.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [7]:
# cpd rigid registration
from probreg import cpd
tf_param, _, _ = cpd.registration_cpd(
    pcd_3dmd_pre, pcd_vicon, 'rigid', maxiter=int(1e5), tol=1e-5,# update_scale=False,
)

r_cab = tf_param.rot
s = tf_param.scale
t = tf_param.t

points_3dmd_cab = tf_param.transform(points_3dmd_pre)

In [8]:
# plot point clouds after calibration
# points_3dmd_cab = obj3d.pcd2np(pcd_3dmd_cab)
pvpcd_3dmd_cab = obj3d.np2pvpcd(points_3dmd_cab, radius=10, phi_resolution=10, theta_resolution=10)

scene = pv.Plotter()
scene.add_mesh(pvpcd_3dmd_cab, color='Gold')
scene.add_mesh(pvpcd_vicon, color='Blue')
scene.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

#### Calibration Validation and Parameter Saving

In [9]:
# combine pre-alignment and registration
# r = np.matmul(r_pre, r_cab)
r = np.matmul(r_cab, r_pre)

In [10]:
# validate the combined parameters
pcd_3dmd_copy = copy.deepcopy(pcd_3dmd)
pcd_3dmd_copy.rotate(r, center=(0, 0, 0))
pcd_3dmd_copy.scale(s, center=(0, 0, 0))
pcd_3dmd_copy.translate(t)

points_3dmd_copy = obj3d.pcd2np(pcd_3dmd_copy)
pvpcd_3dmd_copy = obj3d.np2pvpcd(points_3dmd_copy, radius=10, phi_resolution=10, theta_resolution=10)

scene = pv.Plotter()
scene.add_mesh(pvpcd_3dmd_copy, color='Gold')
scene.add_mesh(pvpcd_vicon, color='Blue')
scene.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [11]:
# save parameters
np.save('../data/calibrate/r', r)
np.save('../data/calibrate/s', s)
np.save('../data/calibrate/t', t)

#### 3dMD-Vicon Mixture System

In [14]:
# plot 3dmd data before calibrating to vicon
mesh = pv.read('../data/6kmh_softbra_8markers_1/speed_6km_soft_bra.000001.obj')
texture = pv.read_texture('../data/6kmh_softbra_8markers_1/speed_6km_soft_bra.000001.jpg')

scene = pv.Plotter()
scene.add_mesh(mesh, show_edges=True)
scene.add_mesh(pvpcd_3dmd, color='Gold')
scene.add_mesh(pvpcd_vicon, color='Blue')
scene.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [15]:
# plot 3dmd data after calibrating to vicon
s, m = obj3d.transform_rst2sm(r, s, t)

mesh_cab = copy.deepcopy(mesh)
mesh_cab.transform(m)
mesh_cab.scale(s)

scene = pv.Plotter()
scene.add_mesh(mesh_cab, show_edges=True)
scene.add_mesh(pvpcd_3dmd_cab, color='Gold')
scene.add_mesh(pvpcd_vicon, color='Blue')
scene.show()

/Users/kirov/Anaconda/anaconda3/lib/python3.8/site-packages/pyvista/core/pointset.py:261: PyvistaDeprecationWarning: You did not specify a value for `inplace` and the default value will be changing to `False` in future versions for point-based meshes (e.g., `PolyData`). Please make sure you are not assuming this to be an inplace operation.
  warnings.warn(DEFAULT_INPLACE_WARNING, PyvistaDeprecationWarning)


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…